In [441]:
import pandas as pd
import logging 
import numpy as np
import re 
from IPython.display import display
from sqlalchemy import MetaData, Table
from database_utils import DatabaseConnector
from data_extraction import DataExtractor
from unidecode import unidecode
from dateutil import parser

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')


In [442]:
# Create an instance of DataExtractor so that I can use it's methods 
instance = DataExtractor()

# Read data from the 'legacy_users' table using the DataExtractor method 
df = instance.read_data_from_table('legacy_users')  
        

read_data_from_table is working
init_db_engine is working
read_db_creds is working


In [443]:
#making a back up of the dataframe 
df_backup = df.copy()

In [444]:
# reset df 
df = df_backup.copy() 

In [445]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15320 entries, 0 to 15319
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   index          15320 non-null  int64 
 1   first_name     15320 non-null  object
 2   last_name      15320 non-null  object
 3   date_of_birth  15320 non-null  object
 4   company        15320 non-null  object
 5   email_address  15320 non-null  object
 6   address        15320 non-null  object
 7   country        15320 non-null  object
 8   country_code   15320 non-null  object
 9   phone_number   15320 non-null  object
 10  join_date      15320 non-null  object
 11  user_uuid      15320 non-null  object
dtypes: int64(1), object(11)
memory usage: 1.4+ MB


In [446]:
# STEP 1 drop null rows 

# Replace 'NULL' with np.nan

num_rows = df.shape[0]
print(f"Number of rows at start: {num_rows}")

#log 
#null_rows = df.loc[df['country_code'] == 'NULL'] 
#display('before cleaning', null_rows)

# Drop rows with any NaN values
df.replace('NULL', np.nan, inplace=True)
df.dropna(inplace=True)

#log 
#null_rows = df.loc[df['country_code'] == 'NULL'] 
#display('after cleaning', null_rows)

#num_rows = df.shape[0]
#print(f"Number of rows after taking out null: {num_rows}")

# drop null values and dupes 

# drop the duplicates from the dataframe directly ('in place')  
df = df.drop_duplicates()

num_rows = df.shape[0]
print(f"Number of rows at end: {num_rows}")


Number of rows at start: 15320
Number of rows at end: 15299


In [447]:
df['country_code'].unique()

array(['DE', 'GB', 'US', 'VSM4IZ4EL3', 'QVUW9JSKY3', 'GGB', '0CU6LW3NKB',
       'PG8MOC0UZI', 'NTCGYW8LVC', 'FB13AKRI21', 'OS2P9CMHR6',
       '5D74J6FPFJ', 'LZGTB0T5Z7', 'IM8MN1L9MJ', 'RVRFD92E48',
       'XKI9UXSCZ1', 'QREF9WLI2A', 'XPVCZE2L8B', '44YAIDY048'],
      dtype=object)

In [448]:
# STEP 2 clean country codes so I have something to base my filtering out of rubbish with 

# replace 'GGB' with 'GB' in the 'country_code' column of the cleaned_country_names_and_codes_df
df['country_code'] = df['country_code'].replace('GGB', 'GB')



In [449]:
df['country_code'].unique()

array(['DE', 'GB', 'US', 'VSM4IZ4EL3', 'QVUW9JSKY3', '0CU6LW3NKB',
       'PG8MOC0UZI', 'NTCGYW8LVC', 'FB13AKRI21', 'OS2P9CMHR6',
       '5D74J6FPFJ', 'LZGTB0T5Z7', 'IM8MN1L9MJ', 'RVRFD92E48',
       'XKI9UXSCZ1', 'QREF9WLI2A', 'XPVCZE2L8B', '44YAIDY048'],
      dtype=object)

In [450]:
# STEP 3 remove rows that are random letters and characters 

regex_country_code = '^[A-Z]{2}$'

dropped_rows = df.loc[~df['country_code'].str.match(regex_country_code)] 
display('before cleaning', dropped_rows)

df = df[df['country_code'].str.match(regex_country_code)] 

dropped_rows = df.loc[~df['country_code'].str.match(regex_country_code)] 
#display('before cleaning', null_rows)
display('after cleaning', dropped_rows)

'before cleaning'

,index,first_name,last_name,date_of_birth,company,email_address,address,country,country_code,phone_number,join_date,user_uuid
752,752,PYCLKLLC7I,W350SCUD6R,KBTI7FI7Y3,R7IZUNSQX0,3Q791B3VIY,YW2YXLOQ5J,I7G4DMDZOZ,VSM4IZ4EL3,A4Q4HQBI3I,JJ2PDVNPRO,W43MSCMQ88
1046,1047,GI4C78KWH0,UTB5PPYFG8,OFH8YGZJWN,CA1XGS8GZW,7HSZB429UK,63GXGYR3XL,AJ1ENKS3QL,QVUW9JSKY3,64ZO0ONUQO,AHN6EKASH3,BUE34OU973
2995,2997,DPAJNJL6PR,B8ZGN8ZJ84,PQPEUO937L,0YJ2FRMDB4,O5Q6D7FDAF,Y1GY1G3EM5,XGI7FM0VBJ,0CU6LW3NKB,DU9UJ42F3E,FYF2FAPZF3,56URKLG01W
3536,3539,E5139W1FJ3,X88YLZX0JX,7KGJ3C5TSW,648S19YUFF,BYU3R2LQF8,GOWRW4JCE7,S0E37H52ON,PG8MOC0UZI,LJ6H5XBD2A,QH6Z9ZPX37,A1W2TVMK1S
5306,5309,XKDVQD7BH2,4YSEX8AY1Z,RQTF5XSXP4,GKJZ58DTHT,789HWCYC7M,KJW5WR7W16,XN9NGL5C0B,NTCGYW8LVC,MUXPJD0BZY,LYVWXBBI6F,LNRNI6X6VL
6420,6426,EI10BU4INS,SH6Y92V6IK,D2OZZHWOLK,LNFBRXXUT4,TFEA83AAHP,XGYW43ZF8B,50KUU3PQUF,FB13AKRI21,GA2YQ189FL,DM4Q84QZ03,23WTQ63AET
8386,8398,I7IJDAPMIM,LUNRZWUKRB,QTVEU5TR8H,JXKOHA5B8F,JK2J6N4DLG,FW3EMRJ25W,EWE3U0DZIV,OS2P9CMHR6,G2D0EI6DT3,DOKMYDVV6L,8FB8XB78NQ
9013,9026,TG6PAO2RAB,PIG6VAZUPM,L3E8OV4UAC,ROL0IMK6UH,EIV7XWUJ8I,KHJLOAKCW2,GMRBOMI0O1,5D74J6FPFJ,65I9CX7VF2,YBUYH8T6OE,HEGG0N3ICG
10211,10224,VCQL3ML6ZT,HULT86HONJ,TLSTUEIKI0,QSS0MWBGFZ,1WNILGM5MK,R4HMRLEIXV,YOTSVPRBQ7,LZGTB0T5Z7,6VGTZ6ANEA,SRH5SM36LH,AKXDQ62K1V
10360,10373,G1SI2RM0KA,67VJEE6DS2,YTC82GP4XE,M4Z6M30SJ3,U1AZKY6URZ,N9D1AYQOJ0,5EFAFD0JLI,IM8MN1L9MJ,Y0AKRC4TUP,4JIOCHZY0W,7JTCX4UVLK


'after cleaning'

,index,first_name,last_name,date_of_birth,company,email_address,address,country,country_code,phone_number,join_date,user_uuid


In [451]:
# STEP 4 clean country names
# regex pattern for filtering out names that contain numbers, special characters (apart from '-')  

num_rows = df.shape[0]
print(f"Number of rows before country cleaning: {num_rows}")

pattern = r'^[a-zA-Z\s-]+$'

#filters the data frame on 2 boolean indexes: the regex boolean index and whether it contains 'NULL' 
df = df[df['country'].str.match(pattern)] # & (df['country'] != 'NULL')]
    
num_rows = df.shape[0]
print(f"Number of rows after country cleaning: {num_rows}")

Number of rows before country cleaning: 15284
Number of rows after country cleaning: 15284


In [452]:
# STEP 5 clean first and last names 
num_rows = df.shape[0]
print(f"Number of rows before name cleaning: {num_rows}")


#convert first_name and last_name  
df['first_name'] = df['first_name'].str.lower()
df['last_name'] = df['last_name'].str.lower()

# DEBUGGING: log that it's worked 
#after_lower_case_example = self.df.iloc[0]['first_name']
#print('this is the name after transformation:', after_lower_case_example)

# strip whitespace from first_name and last_name fields
df['first_name'] = df['first_name'].str.strip()
df['last_name'] = df['last_name'].str.strip()

# Remove special characters using unidecode
df['first_name'] = df['first_name'].apply(unidecode)
df['last_name'] = df['last_name'].apply(unidecode)

regex_pattern = r'^[A-Za-z ._-]+$'

not_applied = df[~df['first_name'].str.match(regex_pattern, na=False)]

if not not_applied.empty:
    print("Names not matching the pattern:")
    print(not_applied)
else:
    print("All names match the pattern.")

num_rows = df.shape[0]
print(f"Number of rows after name cleaning: {num_rows}")


Number of rows before name cleaning: 15284
All names match the pattern.
Number of rows after name cleaning: 15284


In [453]:
display(df.iloc[697]) 

display(df.iloc[3066]) 

display(df.iloc[1]) 

display(df.iloc[2]) 



index                                             697
first_name                                      scott
last_name                                       jones
date_of_birth                              1971/10/23
company                       Jordan, Brown and Evans
email_address                         tom73@jones.net
address            44 Irene plaza\nNew Abbie\nM4H 0NH
country                                United Kingdom
country_code                                       GB
phone_number                            +449098790834
join_date                                  2012-10-08
user_uuid        9ac79783-f8e9-4639-b891-ce6b0d65028a
Name: 697, dtype: object

index                                                      3079
first_name                                                 june
last_name                                             wilkinson
date_of_birth                                        1970-04-19
company                                             Dunn-Thomas
email_address                                 zgraham@ahmed.com
address          Studio 62M\nAbigail forks\nSamuelfurt\nE3H 6US
country                                          United Kingdom
country_code                                                 GB
phone_number                                      0117 496 0803
join_date                                            2012-11-22
user_uuid                  8e7878be-bd04-4ab8-add4-ba230ea3ab96
Name: 3076, dtype: object

index                                                            1
first_name                                                     guy
last_name                                                    allen
date_of_birth                                           1940-12-01
company                                                    Fox Ltd
email_address                         rhodesclifford@henderson.com
address          Studio 22a\nLynne terrace\nMcCarthymouth\nTF0 9GH
country                                             United Kingdom
country_code                                                    GB
phone_number                                       (0161) 496 0674
join_date                                               2001-12-20
user_uuid                     8fe96c3a-d62d-4eb5-b313-cf12d9126a49
Name: 1, dtype: object

index                                               2
first_name                                      harry
last_name                                    lawrence
date_of_birth                              1995-08-02
company                     Johnson, Jones and Harris
email_address            glen98@bryant-marshall.co.uk
address            92 Ann drive\nJoanborough\nSK0 6LR
country                                United Kingdom
country_code                                       GB
phone_number                        +44(0)121 4960340
join_date                                  2016-12-16
user_uuid        fc461df4-b919-48b2-909e-55c95a03fe6b
Name: 2, dtype: object

In [454]:
# STEP 6 clean date of birth 

num_rows = df.shape[0]
print(f"Number of rows before dob cleaning: {num_rows}")


# Initialize a list to store invalid dates
invalid_dates_list = []

# Function to parse dates and standardize format
def parse_date(date_str):
    try:
        # Attempt to parse the date string to a datetime object
        dt = parser.parse(date_str)
        # Convert to the desired format (YYYY-MM-DD)
        return dt.strftime('%Y-%m-%d')
    except (parser.ParserError, ValueError):
        # Append invalid date to the list
        invalid_dates_list.append(date_str)
        return np.nan  # Return NaN for invalid dates

# Apply the function to the 'date_of_birth' column
df['date_of_birth'] = df['date_of_birth'].apply(parse_date)

# Identify rows that would be null after conversion
invalid_rows = df[df['date_of_birth'].isna()]

display("Rows that would be converted to NULL:")
display(invalid_rows)

# Drop rows with NaN (invalid dates)
df_cleaned = df.dropna(subset=['date_of_birth'])

display("\nList of invalid dates:")
display(invalid_dates_list)

num_rows = df.shape[0]
print(f"Number of rows after country cleaning: {num_rows}")


Number of rows before dob cleaning: 15284


'Rows that would be converted to NULL:'

,index,first_name,last_name,date_of_birth,company,email_address,address,country,country_code,phone_number,join_date,user_uuid


'\nList of invalid dates:'

[]

Number of rows after country cleaning: 15284


In [455]:
# STEP 7 clean join date 

num_rows = df.shape[0]
print(f"Number of rows before join date cleaning: {num_rows}")


# Initialize a list to store invalid dates
invalid_join_dates_list = []

# Apply the function to the 'date_of_birth' column
df['join_date'] = df['join_date'].apply(parse_date)

# Identify rows that would be null after conversion
invalid_join_rows = df[df['join_date'].isna()]

display("Rows that would be converted to NULL:")
display(invalid_join_rows)

# Drop rows with NaN (invalid dates)
df_cleaned = df.dropna(subset=['join_date'])

display("\nList of invalid dates:")
display(invalid_join_dates_list)

num_rows = df.shape[0]
print(f"Number of rows after country cleaning: {num_rows}")


Number of rows before join date cleaning: 15284


'Rows that would be converted to NULL:'

,index,first_name,last_name,date_of_birth,company,email_address,address,country,country_code,phone_number,join_date,user_uuid


'\nList of invalid dates:'

[]

Number of rows after country cleaning: 15284


In [456]:
display(df.iloc[697]) 

display(df.iloc[3066]) 

display(df.iloc[1]) 

display(df.iloc[2]) 

index                                             697
first_name                                      scott
last_name                                       jones
date_of_birth                              1971-10-23
company                       Jordan, Brown and Evans
email_address                         tom73@jones.net
address            44 Irene plaza\nNew Abbie\nM4H 0NH
country                                United Kingdom
country_code                                       GB
phone_number                            +449098790834
join_date                                  2012-10-08
user_uuid        9ac79783-f8e9-4639-b891-ce6b0d65028a
Name: 697, dtype: object

index                                                      3079
first_name                                                 june
last_name                                             wilkinson
date_of_birth                                        1970-04-19
company                                             Dunn-Thomas
email_address                                 zgraham@ahmed.com
address          Studio 62M\nAbigail forks\nSamuelfurt\nE3H 6US
country                                          United Kingdom
country_code                                                 GB
phone_number                                      0117 496 0803
join_date                                            2012-11-22
user_uuid                  8e7878be-bd04-4ab8-add4-ba230ea3ab96
Name: 3076, dtype: object

index                                                            1
first_name                                                     guy
last_name                                                    allen
date_of_birth                                           1940-12-01
company                                                    Fox Ltd
email_address                         rhodesclifford@henderson.com
address          Studio 22a\nLynne terrace\nMcCarthymouth\nTF0 9GH
country                                             United Kingdom
country_code                                                    GB
phone_number                                       (0161) 496 0674
join_date                                               2001-12-20
user_uuid                     8fe96c3a-d62d-4eb5-b313-cf12d9126a49
Name: 1, dtype: object

index                                               2
first_name                                      harry
last_name                                    lawrence
date_of_birth                              1995-08-02
company                     Johnson, Jones and Harris
email_address            glen98@bryant-marshall.co.uk
address            92 Ann drive\nJoanborough\nSK0 6LR
country                                United Kingdom
country_code                                       GB
phone_number                        +44(0)121 4960340
join_date                                  2016-12-16
user_uuid        fc461df4-b919-48b2-909e-55c95a03fe6b
Name: 2, dtype: object

IGNORE BELOW THIS LINE 

In [457]:
num_rows = df.shape[0]
print(f"Number of rows before date conversion: {num_rows}")


# Define the regex pattern used in data casting
regex_pattern = r'^\d{4}-\d{2}-\d{2}$'

date_rows_dropped = df[~df['date_of_birth'].str.match(regex_pattern)]

display(date_rows_dropped)
# Convert date_of_birth column to datetime (invalid parsing will be set to NaT)
df['date_of_birth'] = pd.to_datetime(df['date_of_birth'], errors='coerce')

# Identify rows that will be dropped
invalid_rows = df.loc[df['date_of_birth'].isna()]

#display("\nRows that would be converted to null:")
display(invalid_rows)

# Drop rows with NaT (coerced invalid dates)
df = df.dropna(subset=['date_of_birth'])

num_rows = df.shape[0]
print(f"Number of rows after date conversion: {num_rows}")


Number of rows before date conversion: 15284


,index,first_name,last_name,date_of_birth,company,email_address,address,country,country_code,phone_number,join_date,user_uuid


,index,first_name,last_name,date_of_birth,company,email_address,address,country,country_code,phone_number,join_date,user_uuid


Number of rows after date conversion: 15284


In [458]:
display(df['date_of_birth'].unique())

<DatetimeArray>
['1990-09-30 00:00:00', '1940-12-01 00:00:00', '1995-08-02 00:00:00',
 '1972-09-23 00:00:00', '1952-12-20 00:00:00', '1949-08-12 00:00:00',
 '1952-11-10 00:00:00', '2006-08-05 00:00:00', '1959-11-13 00:00:00',
 '1992-10-09 00:00:00',
 ...
 '1944-11-30 00:00:00', '2005-05-06 00:00:00', '1946-07-15 00:00:00',
 '1997-07-21 00:00:00', '1940-09-13 00:00:00', '1959-01-24 00:00:00',
 '1984-11-11 00:00:00', '1961-04-24 00:00:00', '1943-08-09 00:00:00',
 '1994-03-27 00:00:00']
Length: 11334, dtype: datetime64[ns]

In [459]:
# cleaan join date 

self.df['join_date'] = pd.to_datetime(self.df['join_date'], errors='coerce', format='mixed') 

        # DEBUGGING: Check which dates could not be converted
        #invalid_dates = self.df[self.df['join_date'].isna()]
        
        # DEBUGGING: log the number of rows that couldn't be converted in case of errors 
        #print('number of rows that convert to NaT', len(invalid_dates)) 

        # DEBUGGING: Check the conversion worked by printing an example of a difficult date
        #join_date_converted_example = self.df.iloc[202]['join_date']
        #print('original join_date:\n', original_join_date_example)
        #print('join_date after pd.to_datetime:\n', join_date_converted_example)

# dropping rows with NaT values
self.df = self.df.dropna(subset=['join_date'])

NameError: name 'self' is not defined

In [ ]:
#join date 
self.df['join_date'] = pd.to_datetime(self.df['join_date'], errors='coerce', format='mixed') 
